<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 49</b></font><center>
                <center><font size="6">Digital Innovations for Water Challenges</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook 2g</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Spatial data analysis in Google Earth Engine (GEE)</b></font><center></td>
			</tr>
		</table>
		</td>
		<td><center><img src='images/ihe-delft-institute_unesco_fc-lr.jpg'></img></td>
	</tr>
</table>
</div>

# Table of contents
1. [Learning objectives](#learningobs)
2. [Introduction](#introduction)
3. [Installation of required library](#install)
4. [Initializing the GEE](#initial)
5. [Import data](#import)
6. [Clip and display](#clip)
7. [Compute Area](#area)
8. [Image collection and temporal aggregation](#collection)
9. [Zonal statistics and export](#zonal)
10. [Exercise](#exercise)
11. [References and tutorials](#reference)

# 1. Learning objectives<a name="learningobs"></a>

- To access data from Google Earth Engine (GEE) using Python
- To show metadata
- To import data
- To interactively visualise the imported data
- To clip data to study area, compute landcover area
- To perform temporal, univariate and zonal statistics

# 2. Introduction<a name="introduction"></a>

[Google Earth Engine (GEE)](https://earthengine.google.com/) is a computing platform that allows users to run geospatial analysis on Google's infrastructure. It brings together the world's satellite imagery from multiple sources – Landsat, Sentinel, MODIS etc. along with access to other secondary climatic and Geo-physical datasets covering entire globe. GEE has become very popular in the geospatial community and numerous environmental applications at local, regional, and global scales have been deployed all over the globe. GEE provides both JavaScript and Python APIs for making computational requests to the Earth Engine servers. 

Watch [this video](https://youtu.be/gKGOeTFHnKY) for an overview about GEE and what it can do.

In this module we will use [Geemap](https://geemap.org/), which is a Python package for interactive mapping with Google Earth Engine (GEE). It is built upon ipyleaflet and ipywidgets, and enables users to analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment.

Read below paper about Geemap by the developed Prof. Quisheng Wu. <br>
* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
    
<b>Note</b>: You will need a valid login to access GEE. See [this document](docs/Signup_Instructions_GEE.pdf) for instructions to signup.

# 3. Installation of required library<a name="install"></a>
You need to have geemap python library installed in your computer. <br>
If you are using conda environment then use the below command to install geemap. <br>
<b>This is one time step.</b>

In [ ]:
#conda install -c conda-forge geemap

<b>OR</b> use pip to install the geemap package as given below

In [ ]:
#!pip install --upgrade geemap

In [ ]:
#geemap.update_package()

# 4. Initializing the GEE<a name="initial"></a>
Let us start with geemap by importing the library and bringing in a interactive map.

In [ ]:
import os
import ee
import geemap

Now you will define a interactive map and get it inline in this notebook. <br>
<b> Note that first time you do this step in your computer, it will ask for authorize the access needed by GEE. You have to follow the instructions and finish the authorization. This is only one time in local computer</b> <br>
See this [youtube video](https://www.youtube.com/watch?v=h0pz3S6Tvx0&t=359s) showing the authorization steps.

In [ ]:
Map = geemap.Map()
Map

Now that you have successfully initiated the geemap, let us explore data sets from GEE by importing them to this notebook.

# 5. Find and import data in GEE<a name="import"></a>
GEE offers access to petabytes of data from various data sources primarily from satellites and remote sensing.<br>
You can check [this link](https://developers.google.com/earth-engine/datasets) for the data catalog of GEE.<br>
For each type of data, the catalog gives you <b>"Earthengine snippet"</b> as shown below (only an example) which can be used to access this particular data. <br>
<br>
<img src="images/gee1.png" alt="gee" width="500"/>
<br>
**Let us now create a new instance of interactive Map**


In [ ]:
Map = geemap.Map()
Map

Let us import the Copernicus global land cover map of 2019 and display it in the Map above. See details of [Copernicus global land cover map here](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global)

**GEE Functions used** \
[*ee.Image*](https://developers.google.com/earth-engine/apidocs/ee-image): An object to represent an Earth Engine image. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer): Adds a given EE object to the map as a layer.


In [ ]:
landcover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification')
Map.addLayer(landcover, {}, "Land Cover")
# Let us now add the legend of Landcover map to the interactive Map.
Map.add_legend(builtin_legend='ESA_WorldCover')

Let us import the global country boundaries (vector) provided by FAO in GEE and display it in the Map above. See details of [FAO GAUL](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0)

**GEE Functions used** \
[*ee.FeatureCollection*](https://developers.google.com/earth-engine/apidocs/ee-featurecollection): An object to represent an Earth Engine collection of vector, geometry etc. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer): Adds a given EE object to the map as a layer.

In [ ]:
boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
boundaries_level1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
Map.addLayer(boundaries, {}, 'Country Boundaries')

# 6. Clip and display<a name="clip"></a>
GEE offers 'algorithms' to conduct spatial data analysis. The analyses can be performed on the data offered in GEE and also with the user imported data.
Here we are clipping the global landcover map to Kenya and display the country and provinces boundary over the clipped landcover.

**Functions used** \
[*filter*](https://developers.google.com/earth-engine/apidocs/ee-featurecollection-filter): Apply a filter to this collection. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer):  Clips an image to a Geometry or Feature.

In [ ]:
# Below command does a filter based on table schema, here we say "ADM0_NAME == 'Kenya'".
# You will get details of table schema in the data catalog.
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
country = boundaries.filter("ADM0_NAME == 'Kenya'")
provinces = boundaries_level1.filter("ADM0_NAME == 'Kenya'")
Map.addLayer(country, {}, 'Your Country')
# Below command zoom to your country
Map.center_object(country)

Now that we have our Map zoomed into the area of interest and overlayed the boundary of country of interest, let us now clip the global Landcover map to kenya.

**Functions used** \
[*clip*](https://developers.google.com/earth-engine/apidocs/ee-featurecollection-filter): Apply a filter to this collection. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer): Adds a given EE object to the map as a layer.

In [ ]:
landcover_country = landcover.clip(country)
Map.addLayer(landcover_country, {}, 'Country Landcover')
Map.addLayer(provinces, {}, 'Country provinces')

# 7. Compute Area of each landcover type<a name="area"></a>
In this section, let us compute the area of each landcover type in Kenya in Sq.Km

**Functions used** \
[*image_area_by_group*](https://geemap.org/common/#geemap.common.image_area_by_group): Calculates the area of each class of an image.. \

In [ ]:
# Compute area of each land cover type in Sq.Km
df = geemap.image_area_by_group(
    landcover_country, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

# 8. Image collection and temporal aggregation <a name="collection"></a>
In GEE, a stack or time series of images are called Image Collections. GEE offers all time series data related to satellite data, climate, land cover etc as image collections. An ImageCollection can be loaded by pasting an Earth Engine asset ID into the ImageCollection constructor. You can find ImageCollection IDs in the [data catalog](https://developers.google.com/earth-engine/datasets). 
For example, to load the GPM Precipitation data (Rainfall), go to the [product page](https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_MONTHLY_V06) in data catalog and get the asset id given under *Earth Engine Snippet* as shown below.
<br>
<img src="images/gee2.png" alt="gee" width="500"/>
<br>
**Functions used** \
[*ee.ImageCollection*](https://geemap.org/common/#geemap.common.image_area_by_group): Calculates the area of each class of an image. \
[*filterBounds*](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterbounds): Shortcut to filter a collection by intersection with geometry. Items in the collection with a footprint that fails to intersect the given geometry will be excluded. \
[*filterDate*](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate): Shortcut to filter a collection by a date range. \
[select](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-select): Select bands from each image in a collection.
<br>
<br>
**Next, let us bring in monthly GPM precipitation data and filter it to Kenya for the year 2021.** \
**note that the unit is mm/hr**

In [ ]:
pcp_monthly = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06") \
    .filterBounds(country) \
    .filterDate('2021-01-01', '2021-12-31') \
    .select('precipitation')

In the next step we will aggregate the monthly mean precipitation to annual mean and multiply by 8760 (24 * 365 hours in an year) to convert the unit to mm/year. \
Note how two functions can be piped together using '.'
<br>
**Functions used** \
[*mean*](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-mean):  Gets image descriptive statistics. \
[*multiply*](https://developers.google.com/earth-engine/apidocs/ee-image-multiply): Multiplies the first value by the second for each matched pair of bands in image1 and image2. Note this function can be only applied to an image \

In [ ]:
pcp_annual = pcp_monthly.mean().multiply(8760)

Let us now compute the un-variate statistics (mean, minimum, maximum, standard deviation, sum) of the annual precipitation image of Kenya.
<br>
**Functions used** \
[*image_stats*](https://geemap.org/common/#geemap.common.image_stats):  Reduces an image collection by calculating the mean of all values at each pixel across the stack of all matching bands. Bands are matched by name. \
[*getInfo*](https://developers.google.com/earth-engine/apidocs/ee-string-getinfo):  Retrieves the value of this object from the server. \

In [ ]:
pcp_annual_stats = geemap.image_stats(pcp_annual, region=country, scale=10000)
pcp_annual_stats.getInfo()

Let us visualize the annual precipitation. For that we first define a color gradient palette which will vary between min and max value (obtained from last step.)

In [ ]:
pcpViz = {'min': 50, 'max': 1500, 'palette': ['00FFFF', '0000FF']}
Map.addLayer(pcp_annual.clip(country), pcpViz, 'pcp annual')

# 9. Zonal statistics and export image <a name="zonal"></a>
In this section let us extract the descriptive statistics (mean) of annual precipitation for each province in kenya. We will zonal statistics and export the derived statistics into a csv file.
**Functions used** \
[*zonal_statistics*](https://geemap.org/common/#geemap.common.zonal_statistics): Summarizes the values of a raster within the zones of another dataset and exports the results as a csv, shp, json, kml, or kmz. \
[*ee_export_image_to_drive*](https://geemap.org/common/#geemap.common.ee_export_image_to_drive):  Creates a batch task to export an Image as a raster to Google Drive. \

In [ ]:
# First let us define the output folder and file. In this case the folder is 'Downloads' in your user folder.
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_pcp_stats = os.path.join(out_dir, 'pcp_stats.csv')
# Now let us apply zonal statistics and get the output into a csv file
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(pcp_annual, provinces, out_pcp_stats, statistics_type='MEAN', scale=10000)

In the final step download the annual precipitation map for kenya to your google drive in geotif format for further analysis in other GIS softwares like QGIS. \
**Note that depending on the area of interest export can take long time to finish**

In [ ]:
geemap.ee_export_image_to_drive(
    pcp_annual, description='precipitation', folder='gee_output', region=ee.Geometry.bounds(country), scale=10000,
)

# 10. Exercise <a name="exercise"></a>
Now that you have learned how to do spatial analysis in GEE, repeat the same steps (in a new notebook) for your own country and compute:
* Area of each land cover type in hectares
* Average annual rainfall for the year 2020 per each province in your country
* Make a nice interactive Map display showing the landcover, annual rainfall, boundaries

# 11. References and tutorials<a name="reference"></a>
There are multiple tutorials and guides available now online on working with GEE. Below is a list of useful resources on GEE.
<br>
* [geemap tutorials by Prof. Quisheng Wu](https://geemap.org/tutorials/)
* [GEE developer's guide](https://developers.google.com/earth-engine/)
* [GEE API Docs](https://developers.google.com/earth-engine/apidocs)